# Imbalanced-learn for

In [1]:
from sklearn.metrics import classification_report,confusion_matrix,log_loss,auc,plot_confusion_matrix
from sklearn.preprocessing import (
    MinMaxScaler, label_binarize, OneHotEncoder, LabelEncoder)
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFECV
from itertools import cycle, product
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import cycle
import seaborn as sns
import warnings
import random
from tqdm import tqdm_notebook
warnings.filterwarnings('ignore')
%matplotlib inline

## Raw Data

In [2]:
dataset=pd.read_excel('../Case.xlsx',sheet_name='initial')
data=dataset.values

array=[]
for i in data:
    if i[0]==8:
        array.append(list(i))
    data=np.array(array)
    
X =data[:,1:18] #1:18
Y =data[:,19]
seed = random.randint(0,1000)
#seed = 638
X_train,X_test,y_train,y_test = train_test_split(X, Y, test_size=0.2, random_state=seed)

dict_resample = {}
dict_model = {}

## Over-sampling

### RandomOverSample

In [3]:
# 简单的复制样本
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled ,y_resampled= ros.fit_resample(X_train,y_train)
from collections import Counter
print(sorted(Counter(y_resampled).items()))
dict_resample['RandomOverSampler'] = (X_resampled ,y_resampled)

[(0.0, 106), (1.0, 106)]


### SMOTE

In [4]:
from imblearn.over_sampling import SMOTE, ADASYN
X_resampled, y_resampled = SMOTE().fit_resample(X_train, y_train)
print(sorted(Counter(y_resampled).items()))
dict_resample['SMOTE'] = (X_resampled ,y_resampled)

[(0.0, 106), (1.0, 106)]


### ADASYN

In [5]:
from imblearn.over_sampling import SMOTE, ADASYN
X_resampled, y_resampled = ADASYN().fit_resample(X_train, y_train)
print(sorted(Counter(y_resampled).items()))
dict_resample['ADASYN'] = (X_resampled ,y_resampled)

[(0.0, 106), (1.0, 109)]


### BorderlineSMOTE

In [6]:
from imblearn.over_sampling import BorderlineSMOTE
X_resampled, y_resampled = BorderlineSMOTE(kind='borderline-1').fit_resample(X_train, y_train)
print(sorted(Counter(y_resampled).items()))
dict_resample['BorderlineSMOTE-1'] = (X_resampled ,y_resampled)

[(0.0, 106), (1.0, 106)]


In [7]:
from imblearn.over_sampling import BorderlineSMOTE
X_resampled, y_resampled = BorderlineSMOTE(kind='borderline-2').fit_resample(X_train, y_train)
print(sorted(Counter(y_resampled).items()))
dict_resample['BorderlineSMOTE-2'] = (X_resampled ,y_resampled)

[(0.0, 106), (1.0, 106)]


## Over- and under-sampling

### SMOTEENN

In [8]:
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X_train, y_train)
print(sorted(Counter(y_resampled).items()))
dict_resample['SMOTEENN'] = (X_resampled ,y_resampled)

[(0.0, 34), (1.0, 58)]


### SMOTETomek

In [9]:
from imblearn.combine import SMOTETomek
smote_tomek = SMOTETomek(random_state=0)
X_resampled, y_resampled = smote_tomek.fit_resample(X_train, y_train)
print(sorted(Counter(y_resampled).items()))
dict_resample['SMOTETomek'] = (X_resampled ,y_resampled)

[(0.0, 96), (1.0, 96)]


## Bagging

使用训练数据的不同随机子集来训练每个 Base Model，最后每个 Base Model 权重相同，分类问题进行投票，回归问题平均。

随机森林就用到了Bagging，并且具有天然的并行性。

### Bagging

In [10]:
print(sorted(Counter(y_train).items()))
dict_model['Bagging'] = (X_train ,y_train)

[(0.0, 106), (1.0, 27)]


### BalancedBagging

In [11]:
print(sorted(Counter(y_train).items()))
dict_model['BalancedBagging'] = (X_train ,y_train)

[(0.0, 106), (1.0, 27)]


### BalancedRandomForest

In [12]:
print(sorted(Counter(y_train).items()))
dict_model['BalancedRandomForest'] = (X_train ,y_train)

[(0.0, 106), (1.0, 27)]


## Boosting

Boosting是一种迭代的方法，每一次训练会更关心上一次被分错的样本，比如改变被错分的样本的权重的Adaboost方法。还有许多都是基于这种思想，比如Gradient Boosting等。

### RUSBoost

In [13]:
print(sorted(Counter(y_train).items()))
dict_model['RUSBoost'] = (X_train ,y_train)

[(0.0, 106), (1.0, 27)]


### EasyEnsemble

In [14]:
print(sorted(Counter(y_train).items()))
dict_model['EasyEnsemble'] = (X_train ,y_train)

[(0.0, 106), (1.0, 27)]


## 评估

In [15]:
from sklearn.svm import LinearSVC
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
################### train #####################################
scores={}
cm = []
auc = []
for name, (X, y) in tqdm_notebook(list(dict_model.items())):
    if name in ['Bagging']:
        from sklearn.ensemble import BaggingClassifier
        from sklearn.tree import DecisionTreeClassifier
        clf = BaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                random_state=0)
    elif name in ['BalancedBagging']:
        from imblearn.ensemble import BalancedBaggingClassifier
        clf = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                        sampling_strategy='auto',
                                        replacement=False,
                                        random_state=0)
    elif name in ['BalancedRandomForest']:
        from imblearn.ensemble import BalancedRandomForestClassifier
        clf = BalancedRandomForestClassifier(n_estimators=100, random_state=0)
    elif name in ['RUSBoost']:
        from imblearn.ensemble import RUSBoostClassifier
        clf = RUSBoostClassifier(random_state=0)
    elif name in ['EasyEnsemble']:
        from imblearn.ensemble import EasyEnsembleClassifier
        clf = EasyEnsembleClassifier(random_state=0)

    clf.fit(X, y)
    y_test_pred = clf.predict(X_test)
    y_test_score = clf.predict_proba(X_test)  # valid score

    # 混淆矩阵
    cnf_matrix = confusion_matrix(y_test, y_test_pred)
    cm.append((name, cnf_matrix))
    # ROC
    roc_auc = roc_auc_score(
        y_test, y_test_score[:,1])
    auc.append(roc_auc)

  0%|          | 0/5 [00:00<?, ?it/s]

In [16]:
for name, (X, y) in tqdm_notebook(list(dict_resample.items())):
    RANDOM_STATE = 2019
    clf = RandomForestClassifier(n_estimators=161,
                                     max_depth=49,
                                     max_features="sqrt",
                                     random_state=RANDOM_STATE)
    
    clf.fit(X, y)
    y_test_pred = clf.predict(X_test)
    y_test_score = clf.predict_proba(X_test)  # valid score
    
    cnf_matrix = confusion_matrix(y_test, y_test_pred)
    cm.append(('RF'+name, cnf_matrix))
    # ROC
    roc_auc = roc_auc_score(
        y_test, y_test_score[:,1])
    auc.append(roc_auc)

  0%|          | 0/7 [00:00<?, ?it/s]

In [17]:
from sklearn.linear_model import LogisticRegression
for name, (X, y) in tqdm_notebook(list(dict_resample.items())):
    clf = LogisticRegression(penalty="l2",solver="liblinear",C=0.8,max_iter=1000)
       
    clf.fit(X, y)
    y_test_pred = clf.predict(X_test)
    y_test_score = clf.predict_proba(X_test)  # valid score
    
    cnf_matrix = confusion_matrix(y_test, y_test_pred)
    cm.append(('LR'+name, cnf_matrix))
    # ROC
    roc_auc = roc_auc_score(
        y_test, y_test_score[:,1])
    auc.append(roc_auc)

  0%|          | 0/7 [00:00<?, ?it/s]

In [18]:
for name, (X, y) in tqdm_notebook(list(dict_resample.items())):
    clf = AdaBoostClassifier(n_estimators=161,
                             learning_rate=0.5,
                                     random_state=RANDOM_STATE)
    
    clf.fit(X, y)
    y_test_pred = clf.predict(X_test)
    y_test_score = clf.predict_proba(X_test)  # valid score
    
    cnf_matrix = confusion_matrix(y_test, y_test_pred)
    cm.append(('AdaBoost'+name, cnf_matrix))
    # ROC
    roc_auc = roc_auc_score(
        y_test, y_test_score[:,1])
    auc.append(roc_auc)

  0%|          | 0/7 [00:00<?, ?it/s]

In [19]:
for i in auc[1:]:
    print("{:.3f}".format(i))

0.771
0.658
0.619
0.655
0.530
0.577
0.631
0.634
0.557
0.562
0.631
0.708
0.714
0.750
0.768
0.708
0.685
0.738
0.667
0.696
0.649
0.655
0.631
0.577
0.726


In [20]:
for name,c in cm[1:]:
    print("{:.3f}".format(c[1][1]/(c[1][1]+c[0][1]+c[1][0])))

0.308
0.200
0.000
0.182
0.154
0.235
0.286
0.286
0.158
0.125
0.250
0.222
0.235
0.267
0.286
0.235
0.217
0.235
0.182
0.235
0.133
0.188
0.250
0.217
0.333


In [21]:
print(seed)

690


In [22]:
for name,c in cm:
    print("{:}".format(name))

Bagging
BalancedBagging
BalancedRandomForest
RUSBoost
EasyEnsemble
RFRandomOverSampler
RFSMOTE
RFADASYN
RFBorderlineSMOTE-1
RFBorderlineSMOTE-2
RFSMOTEENN
RFSMOTETomek
LRRandomOverSampler
LRSMOTE
LRADASYN
LRBorderlineSMOTE-1
LRBorderlineSMOTE-2
LRSMOTEENN
LRSMOTETomek
AdaBoostRandomOverSampler
AdaBoostSMOTE
AdaBoostADASYN
AdaBoostBorderlineSMOTE-1
AdaBoostBorderlineSMOTE-2
AdaBoostSMOTEENN
AdaBoostSMOTETomek


In [23]:
ytest = pd.Series(y_test) 
ytest.value_counts()

0.0    28
1.0     6
dtype: int64